In [ ]:
# To re-create corpus_data directory
# from harmonic_inference.data.corpus_reading import aggregate_annotation_dfs
# from pathlib import Path

# ANNOTATIONS_PATH = Path('../platti_sonatas')
# OUT_DIR = Path('corpus_data-platti')

# aggregate_annotation_dfs(ANNOTATIONS_PATH, OUT_DIR, notes_only=True)

In [ ]:
from harmonic_inference.data.corpus_reading import load_clean_corpus_dfs

files_df, measures_df, chords_df, notes_df = load_clean_corpus_dfs('corpus_data')

In [ ]:
files_df

In [ ]:
notes_df

In [ ]:
measures_df

In [ ]:
chords_df.columns

In [ ]:
import pandas as pd

def is_major(chord_row):
    if pd.isna(chord_row["relativeroot"]):
        return not chord_row["localkey_is_minor"]
    return chord_row["relativeroot"].split("/")[0][-1].isupper()

chords_df = chords_df.loc[~chords_df["globalkey"].isna()]
    

chords_df.loc[:, "is_major"] = chords_df.apply(is_major, axis=1)

In [ ]:
chords_df.loc[chords_df["is_major"], ["numeral", "chord_type"]]
chords_df.loc[:, ["num:type"]] = chords_df["numeral"] + ":" + chords_df["chord_type"]
chords_df.loc[chords_df["is_major"], "num:type"].value_counts().to_csv("major.tsv", sep="\t")
chords_df.loc[~chords_df["is_major"], "num:type"].value_counts().to_csv("minor.tsv", sep="\t")

In [ ]:
chords_df.loc[(~chords_df["relativeroot"].shift(1).isna()) | (~chords_df["relativeroot"].isna()), ["relativeroot", "numeral", "chord_type"]].to_csv("relative.tsv", sep="\t")

In [ ]:
chords_df.loc[(chords_df["chord_type"] == "Ger") & chords_df["is_major"]]

In [ ]:
"""Training of initial chord prior model"""
import json
from pathlib import Path

import numpy as np

from harmonic_inference.data.piece import Chord
from harmonic_inference.data.data_types import PitchType, KeyMode


initial_chords = chords_df.loc[chords_df.index.get_level_values("chord_id") == 0]
chords = [
    Chord.from_series(row, measures_df.loc[file_id], PitchType.TPC)
    for (file_id, chord_id), row in initial_chords.iterrows()
]

major_key_chords = []
minor_key_chords = []

one_hot_length = chords[0].get_chord_vector_length(
    PitchType.TPC,
    one_hot=True,
    relative=True,
    use_inversions=True,
)
norm_factor = 1 / one_hot_length
major_key_chords_one_hots = np.ones(one_hot_length) * norm_factor
minor_key_chords_one_hots = np.ones(one_hot_length) * norm_factor

for chord in chords:
    one_hot_index = chord.get_one_hot_index(relative=True, use_inversion=True)

    if chord.key_mode == KeyMode.MAJOR:
        major_key_chords.append(chord)
        major_key_chords_one_hots[one_hot_index] += 1
    else:
        minor_key_chords.append(chord)
        minor_key_chords_one_hots[one_hot_index] += 1

# Normalize
major_key_chords_one_hots /= np.sum(major_key_chords_one_hots)
minor_key_chords_one_hots /= np.sum(minor_key_chords_one_hots)

with open(Path("checkpoints", "initial_chord_prior.json"), "w") as json_file:
    json.dump(
        {
            "pitch_type": str(PitchType.TPC).split(".")[1],
            "use_inversions": True,
            "major": list(major_key_chords_one_hots),
            "minor": list(minor_key_chords_one_hots),
        },
        json_file,
        indent=4,
    )

In [ ]:
import harmonic_inference.data.datasets as ds

dataset_classes = [ds.ChordTransitionDataset, ds.ChordClassificationDataset]

dataset_splits = ds.get_dataset_splits(
    files_df,
    measures_df,
    chords_df,
    notes_df,
    dataset_classes,
    splits=[0.8, 0.1, 0.1],
    seed=0,
)

In [ ]:
# Write data out to h5 files
for i1, data_type in enumerate(dataset_classes):
    for i2, split in enumerate(['train', 'valid', 'test']):
        h5_path = Path('h5_data', f'{data_type.__name__}_{split}_seed_{seed}.h5')
        dataset_splits[i1][i2].to_h5(Path(h5_path))

In [ ]:
import eval_utils as eu
import harmonic_utils as hu
import matplotlib.pyplot as plt

label_strings = hu.get_one_hot_labels()
conf_mat = eu.get_conf_mat(labels, outputs)

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()

In [ ]:
import eval_utils as eu

correct, incorrect = eu.get_correct_and_incorrect_indexes(labels, outputs)
print('Correct: ' + str(len(correct)))
print('Incorrect: ' + str(len(incorrect)))

In [ ]:
import eval_utils as eu
    
eu.print_result(incorrect[0], labels, outputs, limit=10, prob=False)

In [ ]:
import eval_utils as eu

chord, onset_notes, all_notes = eu.get_input_df_rows(incorrect[0], datasets[data]['test'])

print(chord)
print("USED NOTES:")
print(onset_notes)
print()
print("ALL NOTES:")
print(all_notes)

In [ ]:
import matplotlib.pyplot as plt
import eval_utils as eu

correct_ranks, indexes_by_rank = eu.get_correct_ranks(labels, outputs)
    
plt.figure(figsize=(30,30))
plt.bar(range(len(outputs[0])), [len(indexes) for indexes in indexes_by_rank])

In [ ]:
import eval_utils as eu
import importlib
importlib.reload(eu)

eval_df = eu.get_eval_df(labels, outputs, datasets[data]['test'])
eval_df

In [ ]:
import ablation
import importlib
importlib.reload(ablation)

dfs = ablation.load_all_ablated_dfs(directory='results', prefix=prefix[:-1] if len(prefix) > 0 else None)
_, mask_names = ablation.get_masks_and_names()

In [ ]:
import pandas as pd
import os

logs = []
for mask_name in mask_names:
    logs.append(pd.read_csv(os.path.join(os.path.join('results', prefix + mask_name + '.log'))))

In [ ]:
for df, log, mask_name in zip(dfs, logs, mask_names):
    print(f"{mask_name} Acc: {100 * df.correct.sum() / len(df)}")
    print(log.iloc[-1])

In [ ]:
import eval_utils as eu

global_df = eu.load_eval_df('results/global_no_ablation.csv')
local_df = eu.load_eval_df('results/local_no_ablation.csv')
none_df = eu.load_eval_df('results/no_ablation.csv')

In [ ]:
global_df

In [ ]:
global_counts = global_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
local_counts = local_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
none_counts = none_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(global_counts['count'], global_counts['mean'], color='red', label='Global key')
plt.scatter(local_counts['count'], local_counts['mean'], color='blue', label='Local key')
plt.scatter(none_counts['count'], none_counts['mean'], color='yellow', label='No transposition')
plt.title('Global key transposed')
plt.xlabel('Count')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Start of baseline evaluation

In [ ]:
from glob import glob
from pathlib import Path
from typing import Tuple
from fractions import Fraction
from bisect import bisect

import pandas as pd
import numpy as np

from harmonic_inference.utils import eval_utils as eu
from harmonic_inference.utils import harmonic_utils as hu
from harmonic_inference.data.data_types import ChordType, PitchType, KeyMode, TRIAD_REDUCTION, ALL_ONE_TYPE_REDUCTION

In [ ]:
results = {}

for file in glob("baseline/*.csv"):
    file_path = Path(file)
    results[file_path.name] = pd.read_csv(file, header=None, names=['on', 'off', 'key', 'degree', 'type', 'inv'])

    # Output is in quarter notes, labels are in whole notes
    results[file_path.name]["on"] /= 4
    results[file_path.name]["off"] /= 4

In [ ]:
keys = set()
degrees = set()
types = set()
inversions = set()

for df in results.values():
    for k in df['key'].unique():
        keys.add(k)
    for d in df['degree'].unique():
        degrees.add(d)
    for t in df['type'].unique():
        types.add(t)
    for i in df['inv'].unique():
        inversions.add(i)

In [ ]:
def key_to_tonic_mode(key: str, pitch_type: PitchType = PitchType.TPC) -> Tuple[int, KeyMode]:
    key = key.replace('-', 'b')
    key = key.replace('+', '#')
    
    tonic = hu.get_pitch_from_string(key, pitch_type)
    mode = KeyMode.MAJOR if key[0].isupper() else KeyMode.MINOR
    
    return tonic, mode

In [ ]:
def type_to_chord_type(type_str: str) -> ChordType:
    return {
        'D7': ChordType.MAJ_MIN7,
        'M': ChordType.MAJOR,
        'd': ChordType.DIMINISHED,
        'd7': ChordType.DIM7,
        'm': ChordType.MINOR,
        'm7': ChordType.MIN_MIN7,
        'Gr+6': ChordType.DIM7,
        'h7': ChordType.HALF_DIM7,
    }[type_str]

In [ ]:
def get_root_tonic_and_mode(
    degree_str: str, tonic: int, mode: KeyMode, pitch_type: PitchType = PitchType.TPC
) -> Tuple[int, int, KeyMode]:
    if isinstance(degree_str, int):
        degree_str = str(degree_str)
        
    degree_str = degree_str.replace('-', 'b')
    degree_str = degree_str.replace('+', '#')
    
    if '/' in degree_str:
        key, degree_str = degree_str.split('/')
        
        relative_transposition = hu.get_interval_from_scale_degree(key, False, mode, pitch_type=pitch_type)
        tonic = hu.transpose_pitch(tonic, relative_transposition, pitch_type=pitch_type)
        
        if key in ['5']:
            mode = KeyMode.MAJOR
        elif key in ['7']:
            mode = KeyMode.MINOR
        elif key in ['1']:
            mode = mode
            
    degree_interval = hu.get_interval_from_scale_degree(degree_str, False, mode, pitch_type=pitch_type)
    root = hu.transpose_pitch(tonic, degree_interval, pitch_type=pitch_type)
    
    return root, tonic, mode

In [ ]:
def get_all(key: str, degree: str, type_str: str, inv: str) -> Tuple[int, ChordType, int, int, KeyMode]:
    inv = int(inv)
    chord_type = type_to_chord_type(type_str)
    
    tonic, mode = key_to_tonic_mode(key)
    root, tonic, mode = get_root_tonic_and_mode(degree, tonic, mode)
    
    return root, chord_type, inv, tonic, mode

In [ ]:
for df in results.values():
    roots = []
    chord_types = []
    invs = []
    tonics = []
    modes = []
    
    for _, row in df.iterrows():
        root, chord_type, inv, tonic, mode = get_all(row['key'], row['degree'], row['type'], row['inv'])
        roots.append(root)
        chord_types.append(chord_type)
        invs.append(inv)
        tonics.append(tonic)
        modes.append(mode)
        
    df["root_tpc"] = roots
    df["chord_type"] = chord_types
    df["inversion"] = invs
    df["tonic"] = tonics
    df["mode"] = modes

In [ ]:
def get_label_df(filename: str) -> pd.DataFrame:
    filename = filename[:-21] + "results.tsv"
    file = glob(f'outputs/**/{filename}', recursive=True)[0]
    
    return pd.read_csv(file, sep='\t', index_col=0, converters={'duration': Fraction}), file

In [ ]:
def get_row_at_onset(df, onset):
    index = min(bisect(list(df['off']), float(onset)), len(df) - 1)
    return df.iloc[index]

In [ ]:
def evaluate_df(key, df):
    label_df, filename = get_label_df(key)
    root_accs = []
    chord_accs = []
    triad_accs = []
    seventh_accs = []
    key_accs = []
    full_accs = []
    
    onset = 0
    for _, label_row in label_df.iterrows():
        est_row = get_row_at_onset(df, onset)
        onset += label_row['duration']
        
        tonic_str = label_row['gt_key'].split(':')[0]
        if '/' in tonic_str:
            tonic_str = tonic_str.split('/')[0]
            
        gt_tonic = hu.get_pitch_from_string(tonic_str, pitch_type=PitchType.TPC)
        gt_mode = KeyMode.MAJOR if label_row['gt_key'][0].isupper() else KeyMode.MINOR
        
        gt_chord = label_row['gt_chord']
        gt_inv = int(gt_chord[-1])
        root_str = gt_chord.split(':')[0]
        if '/' in root_str:
            root_str = root_str.split('/')[0]
        gt_root = hu.get_pitch_from_string(root_str, pitch_type=PitchType.TPC)
        gt_chord_type = hu.get_chord_type_from_string(gt_chord.split(':')[1].split(',')[0])
        
        chord_dist = eu.get_chord_distance(
            gt_root,
            gt_chord_type,
            gt_inv,
            est_row['root_tpc'],
            est_row['chord_type'],
            est_row['inversion'],
        )
        chord_accs.append(1 - chord_dist)

        root_dist = eu.get_chord_distance(
            gt_root,
            gt_chord_type,
            0,
            est_row['root_tpc'],
            est_row['chord_type'],
            0,
            reduction=ALL_ONE_TYPE_REDUCTION
        )
        root_accs.append(1 - root_dist)
        
        triad_dist = eu.get_chord_distance(
            gt_root,
            gt_chord_type,
            0,
            est_row['root_tpc'],
            est_row['chord_type'],
            0,
            reduction=TRIAD_REDUCTION
        )
        triad_accs.append(1 - triad_dist)
        
        seventh_dist = eu.get_chord_distance(
            gt_root,
            gt_chord_type,
            0,
            est_row['root_tpc'],
            est_row['chord_type'],
            0,
        )
        seventh_accs.append(1 - seventh_dist)
        
        key_dist = eu.get_key_distance(
            gt_tonic,
            gt_mode,
            est_row['tonic'],
            est_row['mode'],
        )
        key_accs.append(1 - key_dist)
        
        full_accs.append(1 if chord_dist + key_dist == 0 else 0)
        
    
    root_acc = float(np.average(root_accs, weights=label_df['duration']))
    chord_acc = float(np.average(chord_accs, weights=label_df['duration']))
    key_acc = float(np.average(key_accs, weights=label_df['duration']))
    full_acc = float(np.average(full_accs, weights=label_df['duration']))
    triad_acc = float(np.average(triad_accs, weights=label_df['duration']))
    seventh_acc = float(np.average(seventh_accs, weights=label_df['duration']))
    
    return {
        "Root": root_acc,
        "Triad": triad_acc,
        "Seventh": seventh_acc,
        "Chord": chord_acc,
        "Key": key_acc,
        "Full": full_acc,
    }, filename

In [ ]:
results_vals = {}
import re

for key, df in results.items():
    eval_dict, name = evaluate_df(key, df)

    if not "Beethoven" in name:
        continue
    print(name)
    for acc, val in eval_dict.items():
        if acc not in results_vals:
            results_vals[acc] = []
        results_vals[acc].append(val)
        print(f"    {acc}: {val}")

In [ ]:
for acc, val_list in results_vals.items():
    print(f"{acc}: {sum(val_list) / len(val_list)}")

In [ ]:
from pathlib import Path
from fractions import Fraction

import pandas as pd
from music21.converter import parse

In [ ]:
m21_score = parse(Path("../functional-harmony/data/BPS/scores/bps_01_01.mxl"))
m21_score = m21_score.flattenParts()
m21_score = m21_score.stripTies()

In [ ]:
for note in m21_score.recurse().notes:
    if note.isChord:
        chord = note
        print("Chord")
        for note in chord.notes:
            print(note.pitch.name, note.pitch.octave, chord.duration.quarterLength, chord.offset, chord.measureNumber, note.tie, chord.tie)
        print("End Chord")
    else:
        print(note.offset
        print(note.pitch.name, note.pitch.octave, note.duration.quarterLength, note.offset, note.measureNumber)

In [ ]:
for offset, measure in m21_score.measureOffsetMap().items():
    print(offset, measure[0].timeSignature)

In [ ]:
import importlib
from pathlib import Path
import harmonic_inference.data.piece as piece
importlib.reload(piece)

In [ ]:
notes, measures_df = piece.get_score_piece_from_music_xml(Path("../functional-harmony/data/BPS/scores/bps_01_01.mxl"), "")

In [ ]:
measures_df[40:50]

In [ ]:
list(note for note in notes if note.onset[0] in [48, 49])

# Test loading functional-harmony data

In [ ]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import logging

import harmonic_inference.data.piece as piece

In [ ]:
import importlib
importlib.reload(piece)

In [ ]:
for file_path in tqdm(glob("../functional-harmony/data/**/*.mxl", recursive=True)[173:]):
    music_xml_path = Path(file_path)
    label_csv_path = music_xml_path.parent.parent / "chords" / Path(str(music_xml_path.stem) + ".csv")

    if not label_csv_path.exists():
        logging.error(f"Label file {label_csv_path} does not exist. Skipping.")
        continue

    print(music_xml_path)
    score = piece.get_score_piece_from_music_xml(music_xml_path, label_csv_path)

# Results / Confusion Matrix

In [ ]:
from glob import glob
from tqdm import tqdm
from fractions import Fraction

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from harmonic_inference.data.data_types import PitchType, ChordType, KeyMode
from harmonic_inference.utils.harmonic_utils import get_pitch_from_string, get_chord_type_from_string

In [ ]:
def get_results_df(path):
    dfs = []

    for tsv in tqdm(glob(path, recursive=True)):
        dfs.append(pd.read_csv(tsv, sep="\t", converters={"duration": Fraction}, index_col=0))

    if len(dfs) == 0:
        return None

    results_df = pd.concat(dfs, ignore_index=True)

    for type in ["gt", "est"]:
        results_df[f"{type}_key_tonic"] = 0
        results_df[f"{type}_key_mode"] = 0
        results_df[f"{type}_chord_root"] = 0
        results_df[f"{type}_chord_type"] = 0
        results_df[f"{type}_chord_inv"] = 0

    keys = np.concatenate((results_df["gt_key"].unique(), results_df["est_key"].unique()))

    for key in tqdm(keys, desc="Working on keys..."):
        key_tonic, key_mode = key.split(":")

        for type in ["gt", "est"]:
            results_df.loc[results_df[f"{type}_key"] == key, f"{type}_key_tonic"] = get_pitch_from_string(key_tonic, PitchType.MIDI)
            results_df.loc[results_df[f"{type}_key"] == key, f"{type}_key_mode"] = KeyMode[key_mode.split(".")[1]]

    chords = np.concatenate((results_df["gt_chord"].unique(), results_df["est_chord"].unique()))

    for chord in tqdm(chords, desc="Working on chords..."):
        inv = int(chord[-1])
        chord_str = chord.split(",")[0]
        chord_root, chord_type = chord_str.split(":")

        for type in ["gt", "est"]:
            results_df.loc[results_df[f"{type}_chord"] == chord, f"{type}_chord_root"] = get_pitch_from_string(chord_root, PitchType.TPC)
            results_df.loc[results_df[f"{type}_chord"] == chord, f"{type}_chord_type"] = get_chord_type_from_string(chord_type)
            results_df.loc[results_df[f"{type}_chord"] == chord, f"{type}_chord_inv"] = inv

    return results_df

In [ ]:
results_df = get_results_df("outputs/dcml-csm-1/**/*_results.tsv")
results_df

In [ ]:
def get_heat_map_matrix(results_df):
    heat_map = np.zeros((len(ChordType), len(ChordType) + 2))

    for i, chord_type in tqdm(enumerate(ChordType)):

        chord_type_df = results_df.loc[results_df["gt_chord_type"] == chord_type]
        if len(chord_type_df) == 0:
            continue

        total_dur = float(chord_type_df["duration"].sum())

        correct_root_df = chord_type_df.loc[chord_type_df["gt_chord_root"] == chord_type_df["est_chord_root"]]
        heat_map[i, 0] = float(total_dur - correct_root_df['duration'].sum())

        for j, est_chord_type in enumerate(ChordType, start=1):
            selected_df = correct_root_df.loc[correct_root_df["est_chord_type"] == est_chord_type]

            if est_chord_type == chord_type:
                correct_type_df = selected_df

            selected_dur = float(selected_df["duration"].sum())

            heat_map[i, j] = selected_dur

        if len(correct_type_df) > 0:
            correct_inv_df = correct_type_df.loc[(correct_root_df["gt_chord_inv"] == correct_root_df["est_chord_inv"])]
            heat_map[i, -1] = 1 - float(correct_inv_df['duration'].sum() / correct_type_df['duration'].sum())

    return heat_map

In [ ]:
def normalize_heat_map(heat_map):
    for i, row in enumerate(heat_map):
        if np.sum(row[:-1]) == 0:
            continue
        heat_map[i, :-1] /= np.sum(row[:-1])

In [ ]:
xticks = [
    "M",
    "m",
    "o",
    "+",
    "MM7",
    "d7",
    "mM7",
    "mm7",
    "o7",
    "%7",
    "+7",
    "+M7",
]

In [ ]:
heat_map = get_heat_map_matrix(results_df)

normalize_heat_map(heat_map)

plt.xlabel("Estimated Chord Type", labelpad=-15)
plt.ylabel("Ground Truth Chord Type", rotation=90)
plt.xticks(ticks=np.arange(len(ChordType) + 2), labels=["Incorrect Root"] + xticks + ["Incorrect Inv."], rotation=90)
plt.yticks(ticks=np.arange(len(ChordType)), labels=xticks)
plt.tight_layout(pad=0)
plt.imshow(heat_map, vmin=0, vmax=1)
plt.colorbar()
plt.savefig("figs/heatmap.png", pad_inches=0)

In [ ]:
major_heat_map = get_heat_map_matrix(results_df.loc[results_df["gt_key_mode"] == KeyMode.MAJOR])

normalize_heat_map(major_heat_map)

plt.xlabel("Estimated Chord Type", labelpad=-15)
plt.ylabel("Ground Truth Chord Type", rotation=90)
plt.xticks(ticks=np.arange(len(ChordType) + 2), labels=["Incorrect Root"] + xticks + ["Incorrect Inv."], rotation=90)
plt.yticks(ticks=np.arange(len(ChordType)), labels=xticks)
plt.tight_layout(pad=0)
plt.imshow(major_heat_map, vmin=0, vmax=1)
plt.colorbar()
plt.savefig("figs/heatmap_major.png", pad_inches=0)

In [ ]:
minor_heat_map = get_heat_map_matrix(results_df.loc[results_df["gt_key_mode"] == KeyMode.MINOR])

normalize_heat_map(minor_heat_map)

plt.xlabel("Estimated Chord Type", labelpad=-15)
plt.ylabel("Ground Truth Chord Type", rotation=90)
plt.xticks(ticks=np.arange(len(ChordType) + 2), labels=["Incorrect Root"] + xticks + ["Incorrect Inv."], rotation=90)
plt.yticks(ticks=np.arange(len(ChordType)), labels=xticks)
plt.imshow(minor_heat_map, vmin=0, vmax=1)
plt.colorbar()
plt.tight_layout(pad=0)
plt.savefig("figs/heatmap_minor.png", pad_inches=0)

In [ ]:
def get_acc_given_inversion(results_df, inv):
    inv_df = results_df.loc[results_df["gt_chord_inv"] == inv]

    correct_df = inv_df.loc[inv_df["gt_chord"] == inv_df["est_chord"]]

    return float(correct_df["duration"].sum() / inv_df["duration"].sum())

In [ ]:
results_df = get_results_df("outputs/dcml-csm-1/**/*_results.tsv")
for inv in range(4):
    print(f"Inv {inv} {get_acc_given_inversion(results_df, inv)}")

In [ ]:
def get_acc(results_df):
    total_dur = float(results_df["duration"].sum())
    correct_dur = float(
        results_df.loc[
            (
                (results_df["gt_key"] == results_df["est_key"]) &
                (results_df["gt_chord"] == results_df["est_chord"])
            ),
            "duration",
        ].sum()
    )

    return correct_dur / total_dur

In [ ]:
results_df = get_results_df("outputs/dcml-csm-1/**/*_results.tsv")
acc = get_acc(results_df)
acc_minor = get_acc(results_df.loc[results_df["gt_key_mode"] == KeyMode.MINOR])
acc_major = get_acc(results_df.loc[results_df["gt_key_mode"] == KeyMode.MAJOR])

print(f"Overall: {acc}")
print(f"Minor: {acc_minor}")
print(f"Major: {acc_major}")

In [ ]:
for chord_type in ChordType:
    try:
        print(f"{chord_type}: {get_acc(results_df.loc[results_df['gt_chord_type'] == chord_type])}")
    except:
        pass

In [ ]:
mode_type_heat_map = np.zeros((len(KeyMode), len(ChordType)))

for i, mode in enumerate(KeyMode):
    for j, chord_type in enumerate(ChordType):
        try:
            acc = get_acc(results_df.loc[(results_df['gt_chord_type'] == chord_type) & (results_df['gt_key_mode'] == mode)])
        except:
            continue

        print(f"{mode}, {chord_type} = {acc}")
        mode_type_heat_map[i, j] = acc

In [ ]:
plt.xlabel("Chord Type", fontsize=12)
plt.ylabel("Mode", rotation=90, fontsize=12)
plt.xticks(ticks=np.arange(len(ChordType)), labels=xticks, rotation=90, fontsize=12)
plt.yticks(ticks=np.arange(len(KeyMode)), labels=["Major", "Minor"], fontsize=12)
plt.imshow(mode_type_heat_map, vmin=0, vmax=1)
plt.colorbar(orientation="horizontal", shrink=0.5, pad=0.23)
plt.tight_layout(pad=0)
plt.savefig("figs/acc_by_mode_type.png", pad_inches=0)

In [ ]:
accs = {}

for dir in glob("outputs/dcml-csm-1/*"):
    results_df_comp = get_results_df(dir + "/**/*_results.tsv")
    if results_df_comp is None:
        continue

    accs[dir.split("/")[-1]] = get_acc(results_df_comp)

In [ ]:
for key, value in accs.items():
    print(f"{key}: {value}")

# Converting results TSV to chord-eval comparison for ICMPC

In [ ]:
from tqdm import tqdm
from glob import glob
from pathlib import Path

import pandas as pd

from harmonic_inference.data.data_types import ChordType, PitchType, TRIAD_REDUCTION
from harmonic_inference.utils.harmonic_constants import STRING_TO_CHORD_TYPE
from harmonic_inference.utils.harmonic_utils import get_pitch_from_string, get_pitch_string

In [ ]:
in_path = "outputs/icmpc/*/Mozart-Sonatas/*_results.tsv"

for results_tsv in tqdm(glob(in_path)):
    results_df = pd.read_csv(results_tsv, sep="\t")

    for prefix in ["gt", "est"]:
        results_df[f"{prefix}_chord_root"] = 0
        results_df[f"{prefix}_chord_type"] = 0
        results_df[f"{prefix}_chord_inv"] = 0
    results_df["root_correct"] = 0
    results_df["triad_correct"] = 0
    results_df["7th_correct"] = 0
    results_df["inv_correct"] = 0
    results_df["full_correct"] = 0

    for idx, row in results_df.iterrows():
        gt_root_str, gt_other_str, gt_inv_str = row["gt_chord"].split(":")
        gt_chord_type_str, _ = gt_other_str.split(",")

        gt_root = get_pitch_from_string(gt_root_str, PitchType.MIDI)
        gt_chord_type = STRING_TO_CHORD_TYPE[gt_chord_type_str]

        est_root_str, est_other_str, est_inv_str = row["est_chord"].split(":")
        est_chord_type_str, _ = est_other_str.split(",")

        est_root = get_pitch_from_string(est_root_str, PitchType.MIDI)
        est_chord_type = STRING_TO_CHORD_TYPE[est_chord_type_str]

        results_df.loc[idx, "gt_chord_root"] = gt_root
        results_df.loc[idx, "gt_chord_type"] = str(gt_chord_type)
        results_df.loc[idx, "gt_chord_inv"] = gt_inv_str
        results_df.loc[idx, "est_chord_root"] = est_root
        results_df.loc[idx, "est_chord_type"] = str(est_chord_type)
        results_df.loc[idx, "est_chord_inv"] = est_inv_str

        results_df.loc[idx, "root_correct"] = gt_root == est_root
        results_df.loc[idx, "triad_correct"] = TRIAD_REDUCTION[gt_chord_type] == TRIAD_REDUCTION[est_chord_type]
        results_df.loc[idx, "7th_correct"] = gt_chord_type == est_chord_type
        results_df.loc[idx, "inv_correct"] = gt_inv_str == est_inv_str
        results_df.loc[idx, "full_correct"] = gt_inv_str == est_inv_str and gt_root == est_root and gt_chord_type == est_chord_type

    tsv_path = Path(results_tsv)
    out_path = tsv_path.parent / (tsv_path.name[:-4] + "chord-eval.tsv")
    results_df.to_csv(out_path, sep="\t")